# Análisis Avanzado de Clasificación para Diagnóstico de Cáncer de Mama

Este notebook presenta un análisis exhaustivo y avanzado para la clasificación de tumores de mama utilizando múltiples algoritmos de machine learning, técnicas de optimización y validación robusta.

## Objetivos:
- Realizar un análisis exploratorio profundo del dataset
- Implementar y comparar múltiples algoritmos de clasificación
- Optimizar hiperparámetros usando técnicas avanzadas
- Evaluar modelos con métricas comprehensivas
- Crear visualizaciones interactivas y explicativas

## 1. Importación de Librerías y Configuración

In [ ]:
# Librerías fundamentales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocesamiento y selección de características
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.decomposition import PCA

# Algoritmos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Optimización de hiperparámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

# Métricas de evaluación
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, confusion_matrix, classification_report,
                           roc_curve, precision_recall_curve, average_precision_score)

# Configuración
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Configuración para reproducibilidad
np.random.seed(42)

print('✅ Librerías importadas correctamente')

## 2. Carga y Exploración Inicial de Datos

In [ ]:
# Cargar dataset
df = pd.read_csv('breast-cancer.csv')

print(f'📊 Dimensiones del dataset: {df.shape}')
print(f'🔍 Columnas: {df.columns.tolist()}')

# Información básica
print('\n📋 Información del dataset:')
df.info()

print('\n📈 Primeras 5 filas:')
df.head()

In [ ]:
# Análisis de valores faltantes
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Columna': missing_data.index,
    'Valores_Faltantes': missing_data.values,
    'Porcentaje': missing_percent.values
}).sort_values('Valores_Faltantes', ascending=False)

print('🔍 Análisis de valores faltantes:')
print(missing_df[missing_df['Valores_Faltantes'] > 0])

if missing_df['Valores_Faltantes'].sum() == 0:
    print('✅ No hay valores faltantes en el dataset')

In [ ]:
# Análisis de la variable objetivo
target_counts = df['diagnosis'].value_counts()
target_props = df['diagnosis'].value_counts(normalize=True) * 100

print('🎯 Distribución de la variable objetivo:')
for label, count, prop in zip(target_counts.index, target_counts.values, target_props.values):
    status = 'Maligno' if label == 'M' else 'Benigno'
    print(f'{status} ({label}): {count} casos ({prop:.1f}%)')

# Visualización de la distribución
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico de barras
colors = ['#FF6B6B', '#4ECDC4']
target_counts.plot(kind='bar', ax=ax1, color=colors, alpha=0.8)
ax1.set_title('Distribución de Diagnósticos', fontsize=14, fontweight='bold')
ax1.set_xlabel('Diagnóstico')
ax1.set_ylabel('Número de Casos')
ax1.tick_params(axis='x', rotation=0)

# Gráfico de pastel
ax2.pie(target_counts.values, labels=['Maligno', 'Benigno'], colors=colors, 
        autopct='%1.1f%%', startangle=90)
ax2.set_title('Proporción de Diagnósticos', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 3. Análisis Exploratorio de Datos Avanzado

In [ ]:
# Preparar datos para análisis
# Eliminar columnas no necesarias
if 'id' in df.columns:
    df_analysis = df.drop(['id'], axis=1)
else:
    df_analysis = df.copy()

# Separar características numéricas
numeric_features = df_analysis.select_dtypes(include=[np.number]).columns.tolist()

print(f'📊 Características numéricas encontradas: {len(numeric_features)}')
print(f'🔢 Lista de características: {numeric_features[:10]}...')  # Mostrar solo las primeras 10

In [ ]:
# Estadísticas descriptivas avanzadas
stats_desc = df_analysis[numeric_features].describe()

print('📈 Estadísticas descriptivas:')
print(stats_desc.round(3))

# Análisis de asimetría y curtosis
skewness = df_analysis[numeric_features].skew()
kurtosis = df_analysis[numeric_features].kurtosis()

skew_kurt_df = pd.DataFrame({
    'Característica': numeric_features,
    'Asimetría': skewness.values,
    'Curtosis': kurtosis.values
})

print('\n📊 Análisis de distribución:')
print(skew_kurt_df.round(3))

In [ ]:
# Matriz de correlación avanzada
correlation_matrix = df_analysis[numeric_features].corr()

# Encontrar correlaciones altas
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.8:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                corr_val
            ))

print(f'🔗 Pares de características con alta correlación (>0.8): {len(high_corr_pairs)}')
for feat1, feat2, corr in high_corr_pairs[:10]:  # Mostrar solo los primeros 10
    print(f'  {feat1} - {feat2}: {corr:.3f}')

# Visualización de la matriz de correlación
plt.figure(figsize=(20, 16))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='RdYlBu_r', 
            center=0, square=True, linewidths=0.5)
plt.title('Matriz de Correlación de Características', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de distribuciones por diagnóstico
# Seleccionar algunas características clave para visualización
key_features = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 
               'smoothness_mean', 'compactness_mean']

# Verificar que las características existen
available_features = [feat for feat in key_features if feat in df_analysis.columns]

if len(available_features) < 6:
    # Si no tenemos las características esperadas, usar las primeras 6 numéricas
    available_features = numeric_features[:6]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, feature in enumerate(available_features):
    # Boxplot
    sns.boxplot(data=df_analysis, x='diagnosis', y=feature, ax=axes[i])
    axes[i].set_title(f'Distribución de {feature}', fontweight='bold')
    axes[i].set_xlabel('Diagnóstico')
    
plt.tight_layout()
plt.show()

## 4. Preprocesamiento y Preparación de Datos

In [ ]:
# Preparar características y variable objetivo
X = df_analysis[numeric_features]
y = df_analysis['diagnosis'].map({'M': 1, 'B': 0})  # Maligno=1, Benigno=0

print(f'📊 Forma de X: {X.shape}')
print(f'🎯 Forma de y: {y.shape}')
print(f'✅ Codificación: Maligno=1, Benigno=0')

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'\n📈 Conjunto de entrenamiento: {X_train.shape[0]} muestras')
print(f'📊 Conjunto de prueba: {X_test.shape[0]} muestras')

# Verificar distribución en los conjuntos
print(f'\n🎯 Distribución en entrenamiento:')
print(f'  Maligno: {y_train.sum()} ({y_train.mean()*100:.1f}%)')
print(f'  Benigno: {len(y_train)-y_train.sum()} ({(1-y_train.mean())*100:.1f}%)')

print(f'\n🎯 Distribución en prueba:')
print(f'  Maligno: {y_test.sum()} ({y_test.mean()*100:.1f}%)')
print(f'  Benigno: {len(y_test)-y_test.sum()} ({(1-y_test.mean())*100:.1f}%)')

In [ ]:
# Escalado de características
scalers = {
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler(),
    'MinMaxScaler': MinMaxScaler()
}

# Aplicar diferentes escaladores
scaled_data = {}

for name, scaler in scalers.items():
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    scaled_data[name] = {
        'X_train': X_train_scaled,
        'X_test': X_test_scaled,
        'scaler': scaler
    }

print('✅ Escalado de características completado')
print(f'📊 Escaladores disponibles: {list(scalers.keys())}')

## 5. Selección de Características

In [ ]:
# Selección de características usando diferentes métodos
from sklearn.feature_selection import mutual_info_classif

# Usar StandardScaler para selección de características
X_train_std = scaled_data['StandardScaler']['X_train']
X_test_std = scaled_data['StandardScaler']['X_test']

# 1. Selección univariada (F-score)
selector_f = SelectKBest(score_func=f_classif, k=15)
X_train_f = selector_f.fit_transform(X_train_std, y_train)
X_test_f = selector_f.transform(X_test_std)

# 2. Información mutua
mi_scores = mutual_info_classif(X_train_std, y_train, random_state=42)
mi_features = np.argsort(mi_scores)[-15:]  # Top 15 características

X_train_mi = X_train_std[:, mi_features]
X_test_mi = X_test_std[:, mi_features]

# 3. Eliminación recursiva de características (RFE)
rfe_estimator = LogisticRegression(random_state=42, max_iter=1000)
rfe_selector = RFE(estimator=rfe_estimator, n_features_to_select=15)
X_train_rfe = rfe_selector.fit_transform(X_train_std, y_train)
X_test_rfe = rfe_selector.transform(X_test_std)

print('✅ Selección de características completada')
print(f'📊 Características originales: {X_train_std.shape[1]}')
print(f'🔍 Características seleccionadas: 15')

# Mostrar las características más importantes según F-score
feature_names = X.columns
f_scores = selector_f.scores_
selected_features_f = feature_names[selector_f.get_support()]

print(f'\n🏆 Top 10 características (F-score):')
for i, feat in enumerate(selected_features_f[:10]):
    score_idx = feature_names.get_loc(feat)
    print(f'  {i+1}. {feat}: {f_scores[score_idx]:.2f}')

## 6. Implementación de Múltiples Algoritmos de Clasificación

In [ ]:
# Definir algoritmos de clasificación
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss'),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(random_state=42, max_iter=1000)
}

print(f'🤖 Algoritmos implementados: {len(classifiers)}')
for name in classifiers.keys():
    print(f'  • {name}')

In [ ]:
# Función para evaluar modelos
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Evalúa un modelo y retorna métricas comprehensivas"""
    
    # Entrenar modelo
    model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Métricas
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
    }
    
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    metrics['CV_Mean'] = cv_scores.mean()
    metrics['CV_Std'] = cv_scores.std()
    
    return metrics, y_pred, y_pred_proba

print('✅ Función de evaluación definida')

In [ ]:
# Evaluar todos los modelos con características seleccionadas por F-score
results = []
predictions = {}

print('🚀 Iniciando evaluación de modelos...')
print('=' * 60)

for name, classifier in classifiers.items():
    print(f'📊 Evaluando {name}...')
    
    try:
        metrics, y_pred, y_pred_proba = evaluate_model(
            classifier, X_train_f, X_test_f, y_train, y_test, name
        )
        
        results.append(metrics)
        predictions[name] = {
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }
        
        print(f'  ✅ Accuracy: {metrics["Accuracy"]:.4f}')
        print(f'  📈 F1-Score: {metrics["F1-Score"]:.4f}')
        if metrics['ROC-AUC'] is not None:
            print(f'  🎯 ROC-AUC: {metrics["ROC-AUC"]:.4f}')
        print()
        
    except Exception as e:
        print(f'  ❌ Error: {str(e)}')
        print()

# Crear DataFrame con resultados
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Accuracy', ascending=False)

print('🏆 RESULTADOS FINALES:')
print('=' * 60)
print(results_df.round(4))

## 7. Optimización de Hiperparámetros

In [ ]:
# Seleccionar los mejores modelos para optimización
top_models = results_df.head(3)['Model'].tolist()

print(f'🎯 Optimizando hiperparámetros para los top 3 modelos:')
for i, model in enumerate(top_models, 1):
    print(f'  {i}. {model}')

# Definir grids de hiperparámetros
param_grids = {
    'Logistic Regression': {
        'C': [0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale', 'auto', 0.1, 1]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    },
    'XGBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }
}

In [ ]:
# Optimizar hiperparámetros
optimized_models = {}
optimization_results = []

print('🔧 Iniciando optimización de hiperparámetros...')
print('=' * 60)

for model_name in top_models:
    if model_name in param_grids:
        print(f'⚙️ Optimizando {model_name}...')
        
        # Obtener modelo base
        base_model = classifiers[model_name]
        
        # Grid Search
        grid_search = GridSearchCV(
            base_model,
            param_grids[model_name],
            cv=5,
            scoring='accuracy',
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train_f, y_train)
        
        # Guardar modelo optimizado
        optimized_models[model_name] = grid_search.best_estimator_
        
        # Evaluar modelo optimizado
        opt_metrics, opt_pred, opt_proba = evaluate_model(
            grid_search.best_estimator_, X_train_f, X_test_f, y_train, y_test, 
            f'{model_name} (Optimizado)'
        )
        
        optimization_results.append(opt_metrics)
        
        print(f'  ✅ Mejor score CV: {grid_search.best_score_:.4f}')
        print(f'  📈 Accuracy en test: {opt_metrics["Accuracy"]:.4f}')
        print(f'  🎯 Mejores parámetros: {grid_search.best_params_}')
        print()

# Crear DataFrame con resultados optimizados
if optimization_results:
    opt_results_df = pd.DataFrame(optimization_results)
    opt_results_df = opt_results_df.sort_values('Accuracy', ascending=False)
    
    print('🏆 RESULTADOS DESPUÉS DE OPTIMIZACIÓN:')
    print('=' * 60)
    print(opt_results_df.round(4))

## 8. Visualizaciones Avanzadas y Análisis de Resultados

In [ ]:
# Comparación visual de modelos
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Accuracy comparison
results_df_sorted = results_df.sort_values('Accuracy')
axes[0,0].barh(results_df_sorted['Model'], results_df_sorted['Accuracy'], color='skyblue')
axes[0,0].set_title('Comparación de Accuracy', fontweight='bold')
axes[0,0].set_xlabel('Accuracy')

# 2. F1-Score comparison
results_df_f1 = results_df.sort_values('F1-Score')
axes[0,1].barh(results_df_f1['Model'], results_df_f1['F1-Score'], color='lightcoral')
axes[0,1].set_title('Comparación de F1-Score', fontweight='bold')
axes[0,1].set_xlabel('F1-Score')

# 3. ROC-AUC comparison (solo modelos con probabilidades)
roc_data = results_df.dropna(subset=['ROC-AUC']).sort_values('ROC-AUC')
if not roc_data.empty:
    axes[1,0].barh(roc_data['Model'], roc_data['ROC-AUC'], color='lightgreen')
    axes[1,0].set_title('Comparación de ROC-AUC', fontweight='bold')
    axes[1,0].set_xlabel('ROC-AUC')

# 4. Cross-validation scores
cv_data = results_df.sort_values('CV_Mean')
axes[1,1].barh(cv_data['Model'], cv_data['CV_Mean'], color='gold')
axes[1,1].set_title('Validación Cruzada (Media)', fontweight='bold')
axes[1,1].set_xlabel('CV Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
# Matrices de confusión para los mejores modelos
top_3_models = results_df.head(3)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (_, row) in enumerate(top_3_models.iterrows()):
    model_name = row['Model']
    
    if model_name in predictions:
        y_pred = predictions[model_name]['y_pred']
        cm = confusion_matrix(y_test, y_pred)
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                   xticklabels=['Benigno', 'Maligno'],
                   yticklabels=['Benigno', 'Maligno'])
        axes[i].set_title(f'{model_name}\nAccuracy: {row["Accuracy"]:.3f}', fontweight='bold')
        axes[i].set_xlabel('Predicción')
        axes[i].set_ylabel('Valor Real')

plt.tight_layout()
plt.show()

In [ ]:
# Curvas ROC para modelos con probabilidades
plt.figure(figsize=(12, 8))

colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray']

for i, (model_name, pred_data) in enumerate(predictions.items()):
    if pred_data['y_pred_proba'] is not None:
        fpr, tpr, _ = roc_curve(y_test, pred_data['y_pred_proba'])
        auc_score = roc_auc_score(y_test, pred_data['y_pred_proba'])
        
        plt.plot(fpr, tpr, color=colors[i % len(colors)], 
                label=f'{model_name} (AUC = {auc_score:.3f})', linewidth=2)

# Línea diagonal (clasificador aleatorio)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Clasificador Aleatorio')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)', fontsize=12)
plt.ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=12)
plt.title('Curvas ROC - Comparación de Modelos', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

## 9. Análisis de Importancia de Características

In [ ]:
# Análisis de importancia para Random Forest (si está en top modelos)
if 'Random Forest' in [row['Model'] for _, row in results_df.head(3).iterrows()]:
    # Entrenar Random Forest para obtener importancias
    rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
    rf_model.fit(X_train_f, y_train)
    
    # Obtener importancias
    feature_importance = rf_model.feature_importances_
    selected_feature_names = selected_features_f
    
    # Crear DataFrame de importancias
    importance_df = pd.DataFrame({
        'Característica': selected_feature_names,
        'Importancia': feature_importance
    }).sort_values('Importancia', ascending=False)
    
    # Visualización
    plt.figure(figsize=(12, 8))
    sns.barplot(data=importance_df.head(10), x='Importancia', y='Característica', palette='viridis')
    plt.title('Top 10 Características Más Importantes (Random Forest)', fontsize=14, fontweight='bold')
    plt.xlabel('Importancia')
    plt.tight_layout()
    plt.show()
    
    print('🏆 Top 10 características más importantes:')
    for i, (_, row) in enumerate(importance_df.head(10).iterrows(), 1):
        print(f'  {i}. {row["Característica"]}: {row["Importancia"]:.4f}')

else:
    print('ℹ️ Random Forest no está entre los top 3 modelos para análisis de importancia')

## 10. Análisis de Errores y Casos Límite

In [ ]:
# Análisis de errores del mejor modelo
best_model_name = results_df.iloc[0]['Model']
best_predictions = predictions[best_model_name]

print(f'🎯 Análisis de errores para: {best_model_name}')
print('=' * 50)

# Identificar errores
y_pred_best = best_predictions['y_pred']
errors = y_test != y_pred_best

# Falsos positivos y falsos negativos
false_positives = (y_test == 0) & (y_pred_best == 1)
false_negatives = (y_test == 1) & (y_pred_best == 0)

print(f'📊 Total de errores: {errors.sum()} de {len(y_test)} ({errors.mean()*100:.1f}%)')
print(f'🔴 Falsos positivos: {false_positives.sum()} (Benigno predicho como Maligno)')
print(f'🔴 Falsos negativos: {false_negatives.sum()} (Maligno predicho como Benigno)')

# Análisis de confianza en predicciones
if best_predictions['y_pred_proba'] is not None:
    y_proba_best = best_predictions['y_pred_proba']
    
    # Casos con baja confianza (probabilidad cerca de 0.5)
    low_confidence = np.abs(y_proba_best - 0.5) < 0.1
    
    print(f'\n⚠️ Predicciones con baja confianza: {low_confidence.sum()}')
    print(f'   (Probabilidad entre 0.4 y 0.6)')
    
    # Visualización de distribución de probabilidades
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(y_proba_best[y_test == 0], bins=20, alpha=0.7, label='Benigno (Real)', color='blue')
    plt.hist(y_proba_best[y_test == 1], bins=20, alpha=0.7, label='Maligno (Real)', color='red')
    plt.xlabel('Probabilidad Predicha')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Probabilidades por Clase Real')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.scatter(range(len(y_proba_best)), y_proba_best, 
               c=y_test, cmap='RdYlBu', alpha=0.6)
    plt.axhline(y=0.5, color='black', linestyle='--', alpha=0.5)
    plt.xlabel('Índice de Muestra')
    plt.ylabel('Probabilidad Predicha')
    plt.title('Probabilidades Predichas vs Clase Real')
    plt.colorbar(label='Clase Real (0=Benigno, 1=Maligno)')
    
    plt.tight_layout()
    plt.show()

## 11. Conclusiones y Recomendaciones

In [ ]:
# Resumen final
print('🎯 RESUMEN EJECUTIVO DEL ANÁLISIS')
print('=' * 60)

# Mejor modelo
best_model = results_df.iloc[0]
print(f'🏆 MEJOR MODELO: {best_model["Model"]}')
print(f'   • Accuracy: {best_model["Accuracy"]:.4f} ({best_model["Accuracy"]*100:.1f}%)')
print(f'   • Precision: {best_model["Precision"]:.4f}')
print(f'   • Recall: {best_model["Recall"]:.4f}')
print(f'   • F1-Score: {best_model["F1-Score"]:.4f}')
if best_model['ROC-AUC'] is not None:
    print(f'   • ROC-AUC: {best_model["ROC-AUC"]:.4f}')
print(f'   • CV Score: {best_model["CV_Mean"]:.4f} ± {best_model["CV_Std"]:.4f}')

# Top 3 modelos
print('🥇 TOP 3 MODELOS:')
for i, (_, row) in enumerate(results_df.head(3).iterrows(), 1):
    print(f'   {i}. {row["Model"]}: {row["Accuracy"]:.4f} accuracy')

# Estadísticas del dataset
print('📊 ESTADÍSTICAS DEL DATASET:')
print(f'   • Total de muestras: {len(df)}')
print(f'   • Características originales: {len(numeric_features)}')
print(f'   • Características seleccionadas: 15')
print(f'   • Distribución de clases: {(1-y.mean())*100:.1f}% Benigno, {y.mean()*100:.1f}% Maligno')

# Recomendaciones
print('💡 RECOMENDACIONES:')
print('   1. El modelo seleccionado muestra excelente rendimiento para diagnóstico')
print('   2. Se recomienda validación adicional con datos externos')
print('   3. Considerar implementación en entorno clínico con supervisión médica')
print('   4. Monitorear continuamente el rendimiento del modelo en producción')
print('   5. Actualizar el modelo periódicamente con nuevos datos')

print('✅ ANÁLISIS COMPLETADO EXITOSAMENTE')

## 12. Exportación de Resultados

In [ ]:
# Guardar resultados en archivo CSV
results_df.to_csv('resultados_clasificacion_cancer.csv', index=False)

# Crear reporte detallado
report_data = {
    'Fecha_Analisis': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Dataset_Shape': str(df.shape),
    'Mejor_Modelo': best_model['Model'],
    'Mejor_Accuracy': best_model['Accuracy'],
    'Mejor_F1_Score': best_model['F1-Score'],
    'Caracteristicas_Originales': len(numeric_features),
    'Caracteristicas_Seleccionadas': 15,
    'Modelos_Evaluados': len(classifiers)
}

report_df = pd.DataFrame([report_data])
report_df.to_csv('reporte_analisis_cancer.csv', index=False)

print('💾 Resultados guardados en:')
print('   • resultados_clasificacion_cancer.csv')
print('   • reporte_analisis_cancer.csv')

print('🎉 ¡ANÁLISIS COMPLETO! 🎉')